In [1]:
import torch
import torch.nn as nn    
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np


In [2]:
X,y = load_iris().data, load_iris().target
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = torch.tensor(X_train,dtype=torch.float32)
y_train = torch.tensor(y_train,dtype=torch.long)
X_test = torch.tensor(X_test,dtype=torch.float32)
y_test = torch.tensor(y_test,dtype=torch.long)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

for i,(inputs,labels) in enumerate(train_loader):
    print(f"{inputs.shape}------{labels.shape}")


torch.Size([16, 4])------torch.Size([16])
torch.Size([16, 4])------torch.Size([16])
torch.Size([16, 4])------torch.Size([16])
torch.Size([16, 4])------torch.Size([16])
torch.Size([16, 4])------torch.Size([16])
torch.Size([16, 4])------torch.Size([16])
torch.Size([16, 4])------torch.Size([16])
torch.Size([8, 4])------torch.Size([8])


In [3]:
class IrisNet(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.fc1 = nn.Linear(4,3)
    
    def forward(self,x):
        x = self.fc1(x)
        return x

In [4]:
model = IrisNet()
loss_fcn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=0.01,momentum=0.9)

In [10]:
epochs = 100
correct = 0
total = 0
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    
    for inputs, labels in train_loader:
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        
        _,predicted = torch.max(outputs.data,1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
        train_acc = correct/total * 100
        
        loss = loss_fcn(outputs,labels)
        
        loss.backward()
        optimizer.step()
        running_loss+= loss.item() * inputs.size(0)
        
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch [{epoch + 1}/{epochs}], | Loss: {epoch_loss:.4f}, | training_accuracy: {train_acc:.2f}%')
    


    

Epoch [1/100], | Loss: 0.1416, | training_accuracy: 96.67%
Epoch [2/100], | Loss: 0.1409, | training_accuracy: 96.67%
Epoch [3/100], | Loss: 0.1402, | training_accuracy: 96.67%
Epoch [4/100], | Loss: 0.1394, | training_accuracy: 96.67%
Epoch [5/100], | Loss: 0.1388, | training_accuracy: 96.67%
Epoch [6/100], | Loss: 0.1382, | training_accuracy: 96.67%
Epoch [7/100], | Loss: 0.1376, | training_accuracy: 96.67%
Epoch [8/100], | Loss: 0.1369, | training_accuracy: 96.67%
Epoch [9/100], | Loss: 0.1362, | training_accuracy: 96.67%
Epoch [10/100], | Loss: 0.1357, | training_accuracy: 96.67%
Epoch [11/100], | Loss: 0.1353, | training_accuracy: 96.67%
Epoch [12/100], | Loss: 0.1346, | training_accuracy: 96.67%
Epoch [13/100], | Loss: 0.1339, | training_accuracy: 96.67%
Epoch [14/100], | Loss: 0.1334, | training_accuracy: 96.67%
Epoch [15/100], | Loss: 0.1330, | training_accuracy: 96.67%
Epoch [16/100], | Loss: 0.1324, | training_accuracy: 96.67%
Epoch [17/100], | Loss: 0.1317, | training_accura

In [9]:
# Evaluate the model on the test set
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _ , predicted = torch.max(outputs.data, 1)  # Get the index of the max logit
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
acc = (correct / total) * 100  
print(f'Test Accuracy: {acc:.2f}%')



Test Accuracy: 100.00%
